In [1]:
#For this problem reuse a lot of code from last problem

In [9]:
#SSOR
#Same method but with a forward and backwards SOR

import numpy as np

def SORit(A,u,w,f):
    N = np.size(u)
    s0 = u[0]
    u[0] = (f[0] - np.dot(A[0,1:],u[1:]))/A[0,0]  #First step manually
    u[0] = (1-w)*s0+w*u[0]
    for i in range(1,N-1): #Do N-2 steps automatically following algorithm
        s = u[i]
        u[i] = (f[i]-np.dot(A[i,:i],u[:i])-np.dot(A[i,i+1:],u[i+1:]))/A[i,i]
        u[i] = (1-w)*s+w*u[i]
    sN = u[-1] #Do last step manually
    u[-1] = (f[-1]-np.dot(A[-1,:-1],u[:-1]))/A[-1,-1]
    u[-1] = (1-w)*sN+w*u[-1]
    return u

def SORBWit(A,u,w,f): 
    N = np.size(u)
    sN = u[-1] #Do last step manually
    u[-1] = (f[-1]-np.dot(A[-1,:-1],u[:-1]))/A[-1,-1]
    u[-1] = (1-w)*sN+w*u[-1]
    for i in reversed(range(1,N-1)): #Do N-2 steps automatically following algorithm but reversed
        s = u[i]
        u[i] = (f[i]-np.dot(A[i,:i],u[:i])-np.dot(A[i,i+1:],u[i+1:]))/A[i,i]
        u[i] = (1-w)*s+w*u[i]
    s0 = u[0]
    u[0] = (f[0] - np.dot(A[0,1:],u[1:]))/A[0,0]  #First step manually
    u[0] = (1-w)*s0+w*u[0]
    return u

#SSOR that runs N iterations
def SSOR(A,u,w,f,N):
    u0 = u
    for i in range(N):
        u0 = SORit(A,u0,w,f)
        u0 = SORBWit(A,u0,w,f)
    return u0

#Define SSOR with stopping criteria on recidual vector / source vector (r/f)
def SSORsc(A,u0,w,f,sc,save_residual_lengths):
    u = u0
    r = np.matmul(A,u) - f
    err = np.linalg.norm(r,2) / np.linalg.norm(f,2)
    it = 0  #To count number of iterations
    residuals = [np.linalg.norm(r,2)]
    while err > sc:
        it = it+1
        u = SORit(A,u,w,f)
        u = SORBWit(A,u,w,f)
        f_it = np.matmul(A,u)
        r = f_it - f
        if save_residual_lengths == True:
            residuals.append(np.linalg.norm(r,2))
        err = np.linalg.norm(r,2) / np.linalg.norm(f,2)
        if it > 2000:
            print("Over", it ,"iterations. err is: ",err)
    print("number of iterations :", it)
    return u , np.array(residuals)

In [3]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.sparse.linalg as la
import matplotlib.ticker as ticker

def sourcefunc_2D(x,y):
    f = (x**2 + y**2)*np.sin(x*y)
    return f

def boundary_2D(x,y):
    b = np.sin(x*y)
    return b

def sourcefunc_3D(x,y,z):
    f = ((y**2)*(x**2) + (y**2)*(z**2) + (z**2)*(x**2))*np.sin(x*y*z)
    return f

def boundary_3D(x,y,z):
    b = np.sin(x*y*z)
    return b


In [4]:
def create_Afuex_2D(sourcefunc_2D, boundary_2D, p):

    # Here I create the T_h and I_h matrices. These have precisely the same form as in the lecture notes. Some manual
    # stuff is done since we are working without elimination of boundary conditions
    h = 1/(2**p)
    N = 1/h
    N = int(N)
    Th = sp.diags([-1, 2, -1], [-1, 0, 1], shape=(N-1, N-1)).toarray()
    T_h = np.pad(Th,1)
    T_h[0,0] = h**2
    T_h[N,N] = h**2
    Ih = sp.diags([1],[0], shape=(N-1, N-1)).toarray()
    I_h = np.pad(Ih,1)
    # The final A_h matrix is construced here. Because of the h^2 * I_{N+1} identity matrix in the very top left 
    # corner and bottom right corner I have to change four values manually from zero to 1
    A_2D = (1/(h**2))*np.kron(T_h, I_h) + (1/(h**2))*np.kron(I_h, T_h)
    A_2D[0,0] = 1
    A_2D[N,N] = 1
    A_2D[(N+1)**2-N-1,(N+1)**2-N-1] = 1
    A_2D[(N+1)**2-1,(N+1)**2-1] = 1
    
    
    # A meshgrid is created here on which I will evalute the source function. This vector is the right size for
    # the final result, but it includes every boundary value also, as evaluated through f. This is obviously wrong
    # as these boundary values should be evaluated through b, so that has to be adjusted. I therefore immediately 
    # introduce b1 and b_end as vectors which are the boundary values on the bottom and top of the grid, respectively.
    # f is also reshaped here to be a vector, not an array.
    x,y = np.mgrid[0: 1: complex(0, N+1), 0: 1: complex(0, N+1)]
    x = x.transpose()
    y = y.transpose()

    f_2D = sourcefunc_2D(x,y)
    f_2D = np.reshape(f_2D, (N+1)*(N+1))

    x_axis = np.linspace(0, 1, num = N+1)
    b1 = boundary_2D(x_axis, 0)
    b_end = boundary_2D(x_axis, 1)
    
    # In this section I overwrite the parts of the f vector that represent boundary terms and next-to-boundary terms.
    # In the first loop I overwrite the first and last parts of f with b1 and b_end, so that the bottom and top of the 
    # 'grid' are boundary values. In the second loop I overwrite values representing the left and right side of the
    # 'grid'. Of course the bottom and left boundaries are just filled with zeros, as sin(xy) is zero when either x
    # or y is zero. In the third loop I overwrite the entries which represent positions next to the right boundary. In
    # the last loop I overwrite the entries which represent positions right below the top boundary. 


    for i in range(0, N+1):
        f_2D[i] = b1[i]
        f_2D[(N+1)*N + i] = b_end[i]

    for i in range(1,N):
        f_2D[i*(N+1)] = 0
        f_2D[i*(N+1)+ N] = boundary_2D(1, i*h)
    
    for i in range(0,N-1):    
        f_2D[2*N+i*(N+1)] = f_2D[2*N+i*(N+1)] + boundary_2D(1, (i+1)*h)/(h**2)
    
    for i in range(0,N-1):     
        f_2D[(N+1)**2-1-2*N+i] = f_2D[(N+1)**2-1-2*N+i] + b_end[i+1]/(h**2)
        
    u_ex_pre_2D = boundary_2D(x,y)
    u_ex_2D = np.reshape(u_ex_pre_2D, (1, (N+1)*(N+1)))
    
    
        
    return A_2D , f_2D , u_ex_2D, N

In [6]:
#Create A,f,u_ex for various sizes 2D
data_2D = [0,0]
for p in range(2,8):
    #print(p)
    A_2D , f_2D, u_ex_2D, N = create_Afuex_2D(sourcefunc_2D, boundary_2D, p)
    #A_3D, f_3D, b_3D , u_ex_3D = create_Afuex_3D(A_2D, sourcefunc_3D, boundary_3D, p)
    data_2D.append((A_2D,f_2D,u_ex_2D))

In [8]:
#Solve 2D with SSOR as BIM with stopping criteria 10E-10 and save residuals

from matplotlib import pyplot as plt

stopCrit = 10E-10

resplt2D = []

parr2D = [3,4,5,6]

for i in parr2D:
    Atest = data_2D[i][0]
    ftest = data_2D[i][1]
    u0test = np.zeros(np.size(ftest))   
    #print("A symmetric? :", (Atest == Atest.transpose()).all(),)
    uSSORsc , res = SSORsc(Atest,u0test,1.5,ftest,stopCrit,True)   #Solve with SSOR until stopping criteria. Res is array of two-norms of residuals at each iteration
    resplt2D.append( res ) #Save residual error


number of iterations : 35
number of iterations : 92
number of iterations : 310
Over 1001 iterations. err is:  6.148390502655164e-09
Over 1002 iterations. err is:  6.061069498391515e-09
Over 1003 iterations. err is:  5.974988697253566e-09
Over 1004 iterations. err is:  5.890130388601704e-09
Over 1005 iterations. err is:  5.8064772696669585e-09
Over 1006 iterations. err is:  5.724012220913455e-09
Over 1007 iterations. err is:  5.642718371543995e-09
Over 1008 iterations. err is:  5.562579070516919e-09
Over 1009 iterations. err is:  5.483577926575565e-09
Over 1010 iterations. err is:  5.405698776770782e-09
Over 1011 iterations. err is:  5.328925680055459e-09
Over 1012 iterations. err is:  5.2532429378497546e-09
Over 1013 iterations. err is:  5.178635066357563e-09
Over 1014 iterations. err is:  5.105086767314684e-09
Over 1015 iterations. err is:  5.032583064499157e-09
Over 1016 iterations. err is:  4.961109043324833e-09
Over 1017 iterations. err is:  4.8906501555832176e-09
Over 1018 iterati

In [33]:
#Tabulating the last 5 residual reduction factors

tabs = np.zeros( np.size(resplt2D) )

def return_5_last_residual(residuals):
    for i in range(np.size(resplt2D)):
        restab = np.zeros(5)
        for j in reversed(range(1,6)):
            restab[-j] = resplt2D[i][-j] / resplt2D[i][-(j+1)]    #Taking r_(N-(m)) / r_(N-m-1)
        print(restab)
        tabs[i] = np.array(restab)

for i in range(np.size(resplt2D)):
    restab = np.zeros(5)
    for j in reversed(range(1,6)):
        restab[-j] = resplt2D[i][-j] / resplt2D[i][-(j+1)]    #Taking r_(N-(m)) / r_(N-m-1)
    print("p = ",i+3,restab)


p =  3 [0.57805397 0.57802644 0.57800172 0.57797954 0.57795976]
p =  4 [0.81947334 0.81947336 0.81947338 0.81947337 0.81947335]
p =  5 [0.94600244 0.94600248 0.94600242 0.94600242 0.94600246]
p =  6 [0.98579773 0.98579775 0.98579774 0.98579776 0.98579772]
